# Langevin Dynamics

In [ ]:
import matplotlib.pyplot as plt
import numba
import numpy as np
import scipy.constants as const
from scipy.fft import irfft, rfft
from scipy.stats import linregress

plt.style.use("ggplot")

In [ ]:
# Function definitions for later. Ignore this for now.


def nextpow2(x):
    """
    Calculate the smallest power of 2 greater than or equal to the absolute value
    of the given input.

    Parameters:

    x -- the input number

    Returns:

    n -- the next power of 2
    """
    i = int(abs(x))
    n = 1 if i == 0 else 1 << (i - 1).bit_length()
    return n


@numba.vectorize(
    [numba.float64(numba.complex128)],
    nopython=True,
    fastmath=True,
)
def cabs2(
    x,
):
    """
    Calculate the squared magnitude of a complex number or an array of complex numbers.

    Parameters:

    x -- complex number(s)

    Returns:

    a -- the squared magnitude of the complex number(s)
    """
    a = x.real * x.real + x.imag * x.imag
    return a


def tcf(x):
    """
    Compute the time correlation function (TCF).

    The TCF is an autocorrelation function (ACF) normalized by the decreasing number of samples contributing to the
    average for each lag. The computation is based on the Fast Fourier Transform (FFT).

    Parameters:

    x -- the input signal

    Returns:

    c -- the TCF of the input signal
    """
    nx = x.size

    # Determine FFT length for zero-padding
    nf = nextpow2(2 * nx - 1)

    # Compute FFT, power spectrum, and inverse FFT
    f = rfft(x, nf)
    s = cabs2(f)
    c = irfft(s)

    # Drop negative times.
    c = c[:nx]

    # Normalize by the decreasing number of lag times.
    lags = np.arange(x.size, x.size - c.size, -1)
    c /= lags

    return c


@numba.njit(fastmath=True)
def mssq(x):
    """
    Calculate the time-lagged, mean sum of squares of the input trajectory.

    The mean sum of squares (MSSQ) is defined as
        MSSQ(n) = <x(n)^2 + x(0)^2>
                = SUM(x(n)^2 + x(0)^2) / (LENGTH(x) - n)
                = SSQ(n) / (LENGTH(x) - n)

    The sum of squares (SSQ) is computed efficiently using a recurrence relation,
        SSQ(0) = 2.0 * SUM(x^2)
        SSQ(1) = SSQ(0) - x(0)^2 - x(N-1)^2
        SSQ(2) = SSQ(1) - x(1)^2 - x(N-2)^2
        ...

    Parameters:

    x -- the input trajectory

    Returns:

    s -- the time-lagged, mean sum of squares
    """
    nx = x.size
    xsq = x**2
    s = np.zeros(nx, dtype=np.float64)
    s[0] = 2.0 * np.sum(xsq)
    for n in range(1, nx):
        s[n] = s[n - 1] - xsq[n - 1] - xsq[-n]
    norm = np.arange(x.size, x.size - nx, -1)
    s /= norm
    return s


def msd(trj):
    """
    Compute the mean-squared displacement (MSD) of the input trajectory.

    For an efficient computation, the mean squared displacement
        MSD(n) = <[x(n) - x(0)]^2>
    is separated into
        MSD(n) = MSSQ(n) - 2.0 * TCF(n),
    where
        TCF(n) = <x(n) * x(0)>
    is the time correlation function (TCF), and
        MSSQ(n) = <x(n)^2 + x(0)^2>
    is the time-lagged, mean sum of squares (MSSQ) of the input trajectory.

    The TCF is computed efficiently using the Fast Fourier Transform (FFT), and the mean sum of squares is computed
    using a recurrence relation.

    Parameters:

    x -- the input trajectory

    Returns:

    d -- the mean-squared displacement
    """
    s = mssq(trj)
    c = tcf(trj)
    d = s - 2.0 * c

    return d

## Simulating Free Diffusion

We consider the one-dimensional Langevin equation for a particle undergoing free diffusion.

$$
m\ddot{x} = -\zeta\dot{x} + f_r(t)
$$

1. The following function should integrate the Langevin equation. Fill in the blanks.

In [ ]:
@numba.njit(fastmath=True)
def ld(nsteps, dt, temp, mass, zeta):
    """
    Integrate the 1D Langevin equation using the Euler--Maruyama scheme.

    Parameters:

    nsteps -- the number of simulation steps
    dt -- the time increment
    temp -- the temperature in K
    mass -- the mass of the particle in u
    zeta -- the friction constant in u/ps

    Returns:

    t -- the simulation time
    trj -- the trajectory
    """

    # Initialize time and trajectory arrays.
    t = np.arange(nsteps + 1) * dt
    trj = np.zeros_like(t)

    # Compute the thermal energy in kJ/mol.
    kt = ...

    # Set the initial position to 0.
    x = 0.0

    # Draw a single random number from a standard-normal distribution and scale to obtain a random initial velocity.
    r = np.random.randn(1)[0]
    v = ...

    # Drawn nsteps random numbers from a standard-normal distribution and scale to with the prefactors.
    r = np.random.randn(nsteps)
    r *= ...

    # Implement Euler--Maruyama here.
    for i in range(nsteps):
        ...

    return t, trj

2. Try running a simulation with the following parameters:
   ```
    nsteps = 100000
    dt = 0.001
    temp = 300.0
    mass = 18.0
    zeta = 830.0
    ```
3. Plot the trajectory.
4. Run the simulation multiple times and compare the trajectories.
5. Vary the temperature and the friction constant and observe the effect on the trajectory. Adjust the number of simulation steps and the timestep if necessary.

## Computing Diffusivities

The friction constant and the diffusion constant are related by the fluctuation--dissipation theorem:

$$
D = \frac{kT}{\zeta}
$$

1. Compute the diffusion constant corresponding to $\zeta = 830.0\ \mathsf{u\ ps^{-1}}$. The final value is close to the self diffusion constant of water. Your next job will be recovering this value from your simulation.
2. The function `msd` defined in the beginning computes mean-squared displacements efficiently using Fourier transforms. Don't worry about the implementation unless you are curious. Use this function to compute and plot the mean squared displacement of your trajectory.
3. Is your MSD linear or can you identify a linear regime? Try increasing the number of simulation steps and zooming in on the linear regime by setting explicit x-axis and y-axis limits with `plt.xlim(lower_bound, upper_bound)` and `plt.ylim(lower_bound, upper_bound)`.

In [ ]:
# Compute mean and plot squared displacement.
c = msd(trj)
plt.plot(t, c)
plt.show()

4. Once you have identified a linear regime, we can fit a straight line to it. Before doing so, we have to create arrays `x` and `y`, which contain just the data points in the linear regime. Here is an example of how to do this. Suppose your MSD is linear between $t = 1 \mathsf{ps}$ and $t = 100 \mathsf{ps}$. Since our time step is $\Delta t = 0.001 \mathsf{ps}$, this means the MSD is linear between the steps 1000 and 10000. We can now simply extract the data points in the linear regime by slicing, as shown below. Adapt this to the linear regime you have identified.
5. Verify that your extraction procedure is correct by plotting `x` vs `y`.

In [ ]:
x = t[100:100001]  # Added +1 to the upper bound, because it is not inclusive.
y = c[100:100001]
plt.plot(x, y)
plt.show()

6. Perform a linear fit and recover the slope. Below is how this can be done with Python.
7. Compare with the diffusivity used as simulation input.

In [ ]:
fit = linregress(x, y)
slope = fit.slope
diff = 0.5 * slope

print(f"The diffusivity is {diff:.2g} nm^2 ps^-1")

## Langevin Dynamics in a Harmonic Potential

The Langevin equation can be extended to describe diffusion in a potential,

$$
m\ddot{x} = f_v(x) -\zeta\dot{x} + f_r(t)\,,
$$

where $f_v(x)$ is the systematic force stemming from a potential energy function $V(x)$,

$$
f_v(x) = - \frac{d}{dx}V(x)\,.
$$

In this example, we are going to simulate Langevin dynamics in a harmonic potential, which is a simple model for a harmonic oscillator in a heat bath.

1. Complete the following function, which should integrate Langevin dynamics in a harmonic potential. You can copy most of the code from above and adjust where needed.

In [ ]:
@numba.njit(fastmath=True)
def ld_ho(nsteps, dt, temp, mass, zeta, k):
    """
    Integrate 1D Langevin dynamics in a harmonic potential using the Euler--Maruyama scheme.

    Parameters:

    nsteps -- the number of simulation steps
    dt -- the time increment
    temp -- the temperature in K
    mass -- the mass of the particle in u
    zeta -- the friction constant in u/ps
    k -- the harmonic force constant in kJ/mol/nm^2

    Returns:

    t -- the simulation time
    trj -- the trajectory
    """

    # Initialize time and trajectory arrays.
    t = np.arange(nsteps + 1) * dt
    trj = np.zeros_like(t)

    # Compute the thermal energy in kJ/mol.
    kt = ...

    # Set the initial position to 0.
    x = 0.0

    # Draw a single random number from a standard-normal distribution and scale to obtain a random initial velocity.
    r = np.random.randn(1)[0]
    v = ...

    # Drawn nsteps random numbers from a standard-normal distribution and scale to with the prefactors.
    r = np.random.randn(nsteps)
    r *= ...

    # Implement Euler--Maruyama here.
    for i in range(nsteps):
        ...

    return t, trj

3. Simulate a harmonic oscillator with a force constant $k = 1.0e4\ \mathsf{kJ\ mol^{-1}\ nm^{-2}}$ and $m = 10\ \mathsf{u}$ in a heat bath with $T = 300\ \mathsf{K}$ and $\zeta = 100.0\ \mathsf{u\ ps^{-1}}$.
4. Plot the trajectory.
5. Vary the force constant and the friction constant and observe the effect on the trajectory.
6. Compute the standard deviation of the trajectory data set. What is the theoretical value?
7. Make a histogram of positions. What does the exact probability density look like? Does your simulation match with theory?

## Langevin Dynamics in a Double Well Potential

The function

$$
V(x) = ax^4 - bx^2 \quad (a, b > 0)
$$

defines a simple double well potential.

1. Make a plot of the function with $a = 1\ \mathsf{kJ\ mol^{-1}}$, $b = 4\ \mathsf{kJ\ mol^{-1}}$.
2. Show that the minima are at $x = \pm x_0$ with $x_0 = \sqrt{\frac{b}{2a}}$, and the barrier height is $\Delta{}V = V(0) - V(x_0)=\frac{b^2}{4a}$.
3. Compute the curvature (harmonic force constant) at the well bottom, $k_0 = V^{\prime\prime}(x_0)$, and at the barrier top, $k_b = \left|V^{\prime\prime}(0)\right|$.

4. Implement Langevin Dynamics in a double well potential by completing the following function. Make sure the integration starts in one of the potential wells. 

In [ ]:
@numba.njit(fastmath=True)
def ld_dw(nsteps, dt, temp, mass, zeta, a, b):
    """
    Integrate 1D Langevin dynamics in a double well potential using the Euler--Maruyama scheme.

    Parameters:

    nsteps -- the number of simulation steps
    dt -- the time increment
    temp -- the temperature in K
    mass -- the mass of the particle in u
    zeta -- the friction constant in u/ps
    a -- the parameter a of the double well potential in kJ/mol
    b -- the parameter b of the double well potential in kJ/mol

    Returns:

    t -- the simulation time
    trj -- the trajectory
    """

    # Initialize time and trajectory arrays.
    t = np.arange(nsteps + 1) * dt
    trj = np.zeros_like(t)

    # Compute the thermal energy in kJ/mol.
    kt = ...

    # Set the initial position to one of the potential wells.
    x = ...
    trj[0] = x

    # Draw a single random number from a standard-normal distribution and scale to obtain a random initial velocity.
    r = np.random.randn(1)[0]
    v = ...

    # Drawn nsteps random numbers from a standard-normal distribution and scale to with the prefactors.
    r = np.random.randn(nsteps)
    r *= ...

    # Implement Euler--Maruyama here.
    for i in range(nsteps):
        ...

    return t, trj

5. Run a simulation with $a = 1\ \mathsf{kJ\ mol^{-1}}$, $b = 4\ \mathsf{kJ\ mol^{-1}}$, $m = 10\ \mathsf{u}$, $T = 300\ \mathsf{K}$, and $\zeta = 10.0\ \mathsf{u\ ps^{-1}}$. Plot the trajectory.
6. Make a histogram of the particle positions. Normalize to unit area. What should the exact probability density look like?

Transition state theory (TST) predicts a rate constant for escaping one of the wells:

$$
k_\mathsf{TST} = \frac{\omega_0}{2\pi}e^{-\frac{\Delta{}V}{k_\mathsf{B}T}}\,,
$$

where $\omega_0 = \sqrt{\frac{k_0}{m}}$.

The Kramer's rate is

$$
\begin{align}
\kappa_\mathsf{Kramers} &= \frac{\zeta}{\omega_b}\left(\sqrt{\frac{1}{4}+\frac{\omega_b^2}{\zeta^2}}-\frac{1}{2}\right) \\
k_\mathsf{Kramers} &= \kappa_\mathsf{Kramers}k_\mathsf{TST}\,,
\end{align}
$$

where $\omega_b = \sqrt{\frac{k_b}{m}}$.


7. Compute $k_\mathsf{TST}$ and $k_\mathsf{Kramers}$ for the paramters used in your simulation. Compute the corresponding mean first passage times $\left<t_\mathsf{FP}\right> = k^{-1}$.

8. Run $10^3$ independent simulations for $N=200000$ steps and compute the first passage time for crossing the barrier. Compare with the theoretical results. Feel free to play around with any of the paramters.

In [ ]:
N = 1000
fpt_list = []
for i in range(N):
    t, trj = ld_dw(
        nsteps=200000, dt=0.001, temp=300.0, mass=10.0, zeta=10.0, a=1.0, b=4.0
    )

    # Determine the first passage time.
    mask = trj >= 0
    if mask.any():
        fpt = mask.argmax() * dt
        fpt_list.append(fpt)

mfpt = np.mean(fpt_list)
print(mfpt)